<a href="https://colab.research.google.com/github/Areeb-07/ChatBot/blob/main/Chatter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
with open('/content/drive/MyDrive/Chatbot/data.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [4]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [5]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [6]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 9)                 153       
                                                                 
Total params: 16,697
Trainable params: 16,697
Non-trainable params: 0
____________________________________________________

In [7]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 [==============================] - 1s 9ms/step - loss: 2.1997 - accuracy: 0.0588
Epoch 2/500
2/2 [==============================] - 0s 8ms/step - loss: 2.1967 - accuracy: 0.0392
Epoch 3/500
2/2 [==============================] - 0s 8ms/step - loss: 2.1942 - accuracy: 0.1569
Epoch 4/500
2/2 [==============================] - 0s 8ms/step - loss: 2.1917 - accuracy: 0.1373
Epoch 5/500
2/2 [==============================] - 0s 7ms/step - loss: 2.1897 - accuracy: 0.2353
Epoch 6/500
2/2 [==============================] - 0s 7ms/step - loss: 2.1874 - accuracy: 0.2353
Epoch 7/500
2/2 [==============================] - 0s 6ms/step - loss: 2.1854 - accuracy: 0.2353
Epoch 8/500
2/2 [==============================] - 0s 9ms/step - loss: 2.1835 - accuracy: 0.2353
Epoch 9/500
2/2 [==============================] - 0s 11ms/step - loss: 2.1811 - accuracy: 0.2353
Epoch 10/500
2/2 [==============================] - 0s 8ms/step - loss: 2.1791 - accuracy: 0.2353
Epoch 11/500
2/2 [==========

In [11]:
# to save the trained model
model.save("/content/drive/MyDrive/Chatbot/chat_model")

import pickle

# to save the fitted tokenizer
with open('/content/drive/MyDrive/Chatbot/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL) 
    
# to save the fitted label encoder
with open('/content/drive/MyDrive/Chatbot/label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Chatbot/chat_model/assets


In [14]:
import random
with open("/content/drive/MyDrive/Chatbot/data.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('/content/drive/MyDrive/Chatbot/chat_model')

    # load tokenizer object
    with open('/content/drive/MyDrive/Chatbot/tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('/content/drive/MyDrive/Chatbot/label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print("ChatBot:", np.random.choice(i['responses']))

print("Start messaging with the bot (type quit to stop)!")
chat()

Start messaging with the bot (type quit to stop)!
Hello
ChatBot: Hi
How do you do?
ChatBot: I am fine. Thank you
Good Morning
ChatBot: Good morning
Pleasant meeting you
ChatBot: Bye! Come back again
Pleased meeting you
ChatBot: Have a nice day
Good to meet you
ChatBot: Nice to meet you too
Hola
ChatBot: Hi
Bye Bye
ChatBot: Bye! Come back again
Night
ChatBot: Good afternoon
Wish you a great morning
ChatBot: Good afternoon
Afternoon
ChatBot: I am good. How about you?
How ya doin'?
ChatBot: I am good. How about you?
quit
